## Part 1

In [200]:
import numpy as np
import pandas as pd
import matplotlib as mp
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM

In [201]:
input_table = pd.read_csv(r"C:\Users\gabri\Downloads\midterm_partone.csv")
input_table.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


In [202]:
y_vals  = np.array(input_table["Stock Change"])
x_vals  = np.array(input_table[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()

Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                            gmm   Prob (Hansen J):                 0.729
Method:                           GMM                                         
Date:                Sun, 12 Nov 2023                                         
Time:                        14:16:28                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0200      0.021     -0.964      0.335      -0.061       0.021
p 1            0.0011      0.001      1.843      0.065   -6.89e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
==============================================================================
"""

Now add the delta to the moment conditions of instrumental variables

In [203]:
y_vals  = np.array(input_table["Stock Change"])
x_vals  = np.array(input_table[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm_including_error(GMM):
    def momcond(self, params):
        p0, p1, p2, p3, delta = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] - delta
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] - delta
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] - delta

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])
res = gmm_including_error(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=5).fit(beta0)

res.summary()


Optimization terminated successfully.
         Current function value: 0.000031
         Iterations: 10
         Function evaluations: 15
         Gradient evaluations: 15
Optimization terminated successfully.
         Current function value: 0.000345
         Iterations: 9
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 7
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000346
         Iterations: 2
         Function evaluations: 5
         Gradient evaluations: 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          gmm_including_error Results                          
===============================================================================
Dep. Variable:                       y   Hansen J:                       0.5862
Model:             gmm_including_error   Prob (Hansen J):                 0.444
Method:                            GMM                                         
Date:                 Sun, 12 Nov 2023                                         
Time:                         14:16:28                                         
No. Observations:                 1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0208      0.021     -0.986      0.324      -0.062       0.020
p 1            0.0011      0.001      1.839      0.066   -7.31e-05       0.002
p 2           -0.1062      0.032     -3.316      0.001      -0.169      -0.043
p 3            0.0011      0.000      2.688      0.007       0.000       0.002
p 4           -0.0006      0.003     -0.213      0.831      -0.006       0.005
==============================================================================
"""

## Part 2

Question 1 of Part 2

In [204]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

In [205]:
part_B_data = pd.read_csv(r"C:\Users\gabri\Downloads\midterm_parttwo.csv")

In [206]:
part_B_data.describe(include='all')

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
count,8081.000000,8081,8081,8081,8081,8081,8081
unique,NaN,3,3,5,5,3,2
top,NaN,Low,Less than 2,Very low,Very low,Married,Positive
freq,NaN,5442,3616,3183,6243,5876,6935
mean,2.608588,NaN,NaN,NaN,NaN,NaN,NaN
std,1.571835,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,3.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,3.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [207]:
part_B_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8081 entries, 0 to 8080
Data columns (total 7 columns):
 #   Column                                Non-Null Count  Dtype 
---  ------                                --------------  ----- 
 0   Years of Education after High School  8081 non-null   int64 
 1   Requested Credit Amount               8081 non-null   object
 2   Number of Dependents                  8081 non-null   object
 3   Monthly Income                        8081 non-null   object
 4   Monthly Expense                       8081 non-null   object
 5   Marital Status                        8081 non-null   object
 6   Credit Rating                         8081 non-null   object
dtypes: int64(1), object(6)
memory usage: 442.1+ KB


In [208]:
part_B_data.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative


In [209]:
part_B_data.columns

Index(['Years of Education after High School', 'Requested Credit Amount',
       'Number of Dependents', 'Monthly Income', 'Monthly Expense',
       'Marital Status', 'Credit Rating'],
      dtype='object')

In [210]:
part_B_data = pd.get_dummies(part_B_data, columns=['Requested Credit Amount', 'Number of Dependents', 'Monthly Income', 
                                                   'Monthly Expense', 'Marital Status'], drop_first=True)

In [211]:
X = part_B_data.drop('Credit Rating', axis=1)
y = part_B_data['Credit Rating']

In [212]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

In [213]:
model_classification = LogisticRegression(random_state=0).fit(X_train, y_train)

In [214]:
y_prediction = model_classification.predict(X_test)

In [215]:
c_matrix = confusion_matrix(y_test, y_prediction)
print("Confusion Matrix:")
print(c_matrix)

Confusion Matrix:
[[   0  572]
 [   0 3469]]


In [216]:
recall = recall_score(y_test, y_prediction, pos_label='Positive')
print("Recall: ", recall)

Recall:  1.0


In [217]:
precision = precision_score(y_test, y_prediction, pos_label='Positive')
print("Precision: ", precision)

Precision:  0.8584508784954219


In [218]:
f1 = f1_score(y_test, y_prediction, pos_label='Positive')
print("f1 Score: ", f1)

f1 Score:  0.9238348868175765


Question 2 of Part 2

In [219]:
applications_granted = 0.15
y_probabilities_predicted = model_classification.predict_proba(X_test)[:, 1]

In [220]:
threshold_level = sorted(y_probabilities_predicted)[int((1 - applications_granted) * len(y_probabilities_predicted))]
print(threshold_level)

0.8839365381832269


In [221]:
threshold_accepted_applications = (y_probabilities_predicted >= threshold_level).astype(int)

In [222]:
print(y_test)

6869    Positive
330     Positive
5827    Positive
421     Negative
2357    Positive
          ...   
233     Negative
3272    Positive
178     Positive
283     Positive
4223    Negative
Name: Credit Rating, Length: 4041, dtype: object


In [223]:
y_test_mapping = {'Positive':1, 'Negative':0}
y_test_map = y_test.map(y_test_mapping)

In [224]:
c_matrix = confusion_matrix(y_test_map, threshold_accepted_applications)
print("Confusion Matrix with Threshold:")
print(c_matrix)

Confusion Matrix with Threshold:
[[ 503   69]
 [2931  538]]


In [225]:
recall = recall_score(y_test_map, threshold_accepted_applications, pos_label=1)
print("Recall with Threshold: ", recall)

Recall with Threshold:  0.15508792159123666


In [226]:
precision = precision_score(y_test_map, threshold_accepted_applications, pos_label=1)
print("Precision with Threshold: ", precision)

Precision with Threshold:  0.886326194398682


In [227]:
f1 = f1_score(y_test_map, threshold_accepted_applications, pos_label=1)
print("f1 Score with Threshold: ", f1)

f1 Score with Threshold:  0.26398429833169773
